In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

Аналитическое решение модельной задачи

In [2]:
def e(x):             # функция экпоненты
    return np.exp(x)


x0 = 1/np.sqrt(2)     # условия задачи
u0 = 1
u1 = 0


k_a = 1               # константы в формуле ан. решения
q_a = 1
f_a = e(x0)
k_b = e(np.sin(x0))
q_b = 2
f_b = e(x0)

lambda_a = np.sqrt(q_a/k_a)
lambda_b = np.sqrt(q_b/k_b)

m_a = f_a/q_a
m_b = f_b/q_b

a11 = e(-lambda_a*x0) - e(lambda_a*x0)
a12 = e(lambda_b*(2-x0)) - e(lambda_b*x0)
a21 = k_a*lambda_a*(e(lambda_a*x0) + e(-lambda_a*x0))
a22 = k_b*lambda_b*(e(lambda_b*(2-x0)) + e(lambda_b*x0))

b1 = m_b - m_a + (m_a - u0)*e(lambda_a*x0) - (m_b - u1)*e(lambda_b*(1 - x0))
b2 = k_a*lambda_a*(u0 - m_a)*e(lambda_a*x0) + k_b*lambda_b*(u1 - m_b)*e(lambda_b*(1 - x0))

c1 = (((u0 - m_a)*a11 - b1)*a22 - ((u0 - m_a)*a21 - b2)*a12) / (a11*a22 - a12*a21)
c2 = (b1*a22 - b2*a12) / (a11*a22 - a12*a21)
c3 = (b2*a11 - b1*a21) / (a11*a22 - a12*a21)
c4 = (u1 - m_b)*e(lambda_b) - c3*e(2*lambda_b)


def u_model(N):  # выводит значение аналитического решения в точке х
    y = []
    x = np.linspace(0, 1, N)
    for i in range(N):
        if x[i] < x0: 
            y.append(c1*e(lambda_a*x[i]) + c2*e(-lambda_a*x[i]) + m_a)
        if x[i] > x0:
            y.append(c3*e(lambda_b*x[i]) + c4*e(-lambda_b*x[i]) + m_b)
    return np.array(y)

Численное решение модельной задачи

In [3]:
# коэффициенты в формулах для прогоночных коэффициентов 
def abcd_model(l, l_alpha, l_betta, h, L):
    
    if l >= 1 and l <= l_alpha - 1:
        a = k_a
        b = - 2*k_a - q_a*h*h
        c = k_a
        d = - f_a*h*h
    if l <= L - 1 and l >= l_betta + 1:
        a = k_b
        b = - 2*k_b - q_b*h*h
        c = k_b
        d = - f_b*h*h 
        
    return [a, b, c, d]


# выводит массив значений численного решения в в N узлах отрезка [0, 1]
def solver_model(N):
    
    L = N - 1                 # номер последнего узла
    x = np.linspace(0, 1, N)  # массив значений х в N узлах
    u = np.zeros(N)           # массив значений u
    h = 1/(N-1)               # шаг по х
    
    
    for i in range(L + 1):
        if x[i] < x0 and x[i+1] > x0:
            l_alpha = i       # узел слева от х0
            l_betta = i + 1   # узел cправа от х0 
      
    
    # прогоночные коэффициенты
    alpha = np.zeros(N)
    betta = np.zeros(N)
    
    alpha[1] = - k_a / (- 2*k_a - q_a*h*h)
    alpha[L-1] = - k_b / (- 2*k_b - q_b*h*h)
    
    betta[1] = ((- f_a*h*h) - k_a*u0) / (- 2*k_a - q_a*h*h) 
    betta[L-1] = ((- f_b*h*h) - k_b*u1) / (- 2*k_b - q_b*h*h) 
    
    k = abcd_model(2, l_alpha, l_betta, h, L)
    for l in range(2, l_alpha):
        alpha[l] =  - k[0] / (k[1] + k[2]*alpha[l - 1])
        betta[l] = (k[3] - k[2]*betta[l - 1]) / (k[1] + k[2]*alpha[l - 1])
       
    k = abcd_model(l_betta + 1, l_alpha, l_betta, h, L)
    for l in reversed(range(l_betta + 1, L - 1)):
        alpha[l] =  - k[2] / (k[1] + k[0]*alpha[l + 1])
        betta[l] = (k[3] - k[0]*betta[l + 1]) / (k[1] + k[0]*alpha[l + 1])
        
        
    # краевые условия
    u[0] = u0
    u[L] = u1
    
    # условия разрыва
    u[l_alpha] = (k_a*betta[l_alpha - 1] + k_b*betta[l_betta + 1]) / (k_a*(1 - alpha[l_alpha - 1]) + k_b*(1 - alpha[l_betta + 1]))
    u[l_betta] = u[l_alpha]
    u[l_alpha - 1] = alpha[l_alpha - 1]*u[l_alpha] + betta[l_alpha - 1]
    u[l_betta + 1] = alpha[l_betta + 1]*u[l_betta] + betta[l_betta + 1]
    
    
    # обратная прогонка
    for l in reversed(range(1, l_alpha - 1)):
        u[l] = alpha[l]*u[l + 1] + betta[l]
    for l in range(l_betta + 2, L):
        u[l] = alpha[l]*u[l - 1] + betta[l]
    
    return u

Численное решение задачи с переменными коэффициентами

In [42]:
# коэффициенты в формулах для прогоночных коэффициентов 
def abcd_alt(l, l_alpha, l_betta, h, L, x_fict):
    
    if l >= 1 and l <= l_alpha - 1:
        a = 1
        b = - (2 + q_a*h*h)
        c = 1
    if l <= L - 1 and l >= l_betta + 1:
        a = e(np.sin(x_fict[2*l + 1]))
        b = - (e(np.sin(x_fict[2*l + 1])) + e(np.sin(x_fict[2*l - 1])) + q_b*h*h)
        c = e(np.sin(x_fict[2*l - 1]))
    d = - e(x_fict[2*l])*h*h
    return [a, b, c, d]

def solver_alt(N):
    L = N - 1                      # номер последнего узла
    M = 2*N - 1                    # номер последнего узла в удвоенной сетке
    x = np.linspace(0, 1, N)       # массив значений х в N узлах
    x_fict = np.linspace(0, 1, M)  # фиктивный массив
    u = np.zeros(N)                # массив значений u
    h = 1/(N - 1)                  # шаг по х


    for i in range(L + 1):
        if x[i] < x0 and x[i+1] > x0:
            l_alpha = i            # узел слева от х0
            l_betta = i + 1        # узел cправа от х0 
    
    alpha = np.zeros(N)
    betta = np.zeros(N)
    
    alpha[1] = - 1 / (- (2 + q_a*h*h))
    alpha[L-1] = - e(np.sin(x_fict[2*(L-1) + 1])) / (- (e(np.sin(x_fict[2*(L-1) + 1])) + e(np.sin(x_fict[2*(L-1) - 1])) + q_b*h*h))
    
    betta[1] = ((- e(x_fict[2])*h*h) - u0) / (- (2 + q_a*h*h)) 
    betta[L-1] = ((- e(x_fict[2*(L-1)])*h*h) - e(np.sin(x_fict[2*(L-1) + 1]))*u1) / (- (e(np.sin(x_fict[2*(L-1) + 1])) + e(np.sin(x_fict[2*(L-1) - 1])) + q_b*h*h))  
    
    for l in range(2, l_alpha):
        k = abcd_alt(l, l_alpha, l_betta, h, L, x_fict)
        alpha[l] =  - k[0] / (k[1] + k[2]*alpha[l - 1])
        betta[l] = (k[3] - k[2]*betta[l - 1]) / (k[1] + k[2]*alpha[l - 1])
        
    for l in reversed(range(l_betta + 1, L - 1)):
        k = abcd_alt(l, l_alpha, l_betta, h, L, x_fict)
        alpha[l] =  - k[2] / (k[1] + k[0]*alpha[l + 1])
        betta[l] = (k[3] - k[0]*betta[l + 1]) / (k[1] + k[0]*alpha[l + 1])

    k_a = 1
    k_b = e(np.sin(x[l_alpha]))

    u[0] = u0
    u[L] = u1
    u[l_alpha] = (k_a*betta[l_alpha - 1] + k_b*betta[l_betta + 1]) / (k_a*(1 - alpha[l_alpha - 1]) + k_b*(1 - alpha[l_betta + 1]))
    u[l_betta] = u[l_alpha]
    u[l_alpha - 1] = alpha[l_alpha - 1]*u[l_alpha] + betta[l_alpha - 1]
    u[l_betta + 1] = alpha[l_betta + 1]*u[l_betta] + betta[l_betta + 1]

    for l in reversed(range(1, l_alpha - 1)):
        u[l] = alpha[l]*u[l + 1] + betta[l]
    for l in range(l_betta + 2, L):
        u[l] = alpha[l]*u[l - 1] + betta[l]
    return u

def output(N):
    print('число узлов: ' + str(N))
    x_output = np.linspace(0, 1, 11)
    array = np.zeros((11, 4))
    h = 1/(N-1)
    for i in range(11):
        array[i][0] = u_model(N)[int(round(x_output[i]/h))]
        array[i][1] = solver_model(N)[int(round(x_output[i]/h))]
        array[i][2] = np.fabs(array[i][1] - array[i][0])
        array[i][3] = solver_alt(N)[int(round(x_output[i]/h))]
        stlb_name = ['ANALYTICAL', 'MODEL', 'DIFFERENCE', 'ALTERNATING']
    return pd.DataFrame(array, x_output, stlb_name)

In [45]:
output (1001)

число узлов: 1001


,ANALYTICAL,MODEL,DIFFERENCE,ALTERNATING
0.0,1.000000e+00,1.000000,0.000000e+00,1.000000
0.1,9.349441e-01,0.934803,1.416053e-04,0.910006
0.2,8.589475e-01,0.858663,2.846270e-04,0.818050
0.3,7.712494e-01,0.770819,4.304963e-04,0.722047
0.4,6.709721e-01,0.670391,5.806731e-04,0.619750
0.5,5.571121e-01,0.556375,7.366603e-04,0.508711
0.6,4.285299e-01,0.427630,9.000191e-04,0.386250
0.7,2.839384e-01,0.282866,1.072384e-03,0.249402
0.8,1.947039e-01,0.194568,1.358895e-04,0.167762
0.9,1.021185e-01,0.102051,6.759186e-05,0.086510


In [46]:
output(10001)

число узлов: 10001


,ANALYTICAL,MODEL,DIFFERENCE,ALTERNATING
0.0,1.000000e+00,1.000000,0.000000e+00,1.000000
0.1,9.349441e-01,0.934930,1.383978e-05,0.910108
0.2,8.589475e-01,0.858920,2.781813e-05,0.818254
0.3,7.712494e-01,0.771207,4.207493e-05,0.722355
0.4,6.709721e-01,0.670915,5.675287e-05,0.620165
0.5,5.571121e-01,0.557040,7.199884e-05,0.509238
0.6,4.285299e-01,0.428442,8.796542e-05,0.386894
0.7,2.839384e-01,0.283834,1.048124e-04,0.250170
0.8,1.947039e-01,0.194690,1.386425e-05,0.167774
0.9,1.021185e-01,0.102112,6.896081e-06,0.086515


In [47]:
output(100001)

число узлов: 100001


,ANALYTICAL,MODEL,DIFFERENCE,ALTERNATING
0.0,1.000000e+00,1.000000,0.000000e+00,1.000000
0.1,9.349441e-01,0.934942,1.884385e-06,0.910117
0.2,8.589475e-01,0.858944,3.786818e-06,0.818273
0.3,7.712494e-01,0.771244,5.726302e-06,0.722384
0.4,6.709721e-01,0.670964,7.722401e-06,0.620203
0.5,5.571121e-01,0.557102,9.795224e-06,0.509287
0.6,4.285299e-01,0.428518,1.196559e-05,0.386953
0.7,2.839384e-01,0.283924,1.425535e-05,0.250240
0.8,1.947039e-01,0.194703,9.250627e-07,0.167776
0.9,1.021185e-01,0.102118,4.601708e-07,0.086516
